<a href="https://colab.research.google.com/github/kimminju99/tsp_using_genetic_algorithm/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
file_uploaded = files.upload()

Saving TSP.csv to TSP.csv


In [5]:
import random,copy
import numpy as np
import csv
import pandas as pd

# given cities
cities =[]
len_cities = 1000
matrix = [[0 for _ in range(len_cities)] for _ in range(len_cities)]
visitied = [False for _ in range(len_cities)]

# Euclidean distance measuring function
def distance(x,y):
    dist = np.linalg.norm(np.array(x)-np.array(y))
    return dist

# get TSP city map
with open('TSP.csv', mode ='r', newline ='') as tsp:
    # read TSP city map
    reader = csv.reader(tsp)
    for row in reader:
        cities.append(row)

def eval_matrix():
    global matrix, cities

    for i in range(len_cities):
        for j in range(len_cities):
            if i == j:
                matrix[i][j] = 0
            elif matrix[i][j] == 0:
                city_1 = [float(cities[i][0]),float(cities[i][1])]
                city_2 = [float(cities[j][0]),float(cities[j][1])]
                matrix[i][j] = distance(city_1, city_2)
                matrix[j][i] = matrix[i][j]

def tree(start):
    global matrix, visited
    chrom = []
    chrom.append(start)
    visited[start] = True
    
    for _ in range(len_cities-1):
        city = matrix[start].index(min(list(value for value in matrix[start] if visited[matrix[start].index(value)] == False and value != 0)))
        visited[city] = True
        chrom.append(city)
        start = city

    return chrom

# 초기 염색체 트리
def init_population_tree(size, C):
    global visited
    population = []
    random.seed(0)

    root = list(random.sample(range(0,len_cities), size))
    for start in root:
        visited = [False for _ in range(len_cities)]
        new_chrom = tree(start)
        fit = pow(C/eval_fit_distance(new_chrom), 2)
        population.append((new_chrom, fit))

    return  population

# 초기 염색체
def init_population_random(size, C):
    population = []
    random.seed(0)

    for _ in range(size):
        new_chrom = list(range(0, len_cities))
        random.shuffle(new_chrom)
        fit = pow(C/eval_fit_distance(new_chrom), 2)
        population.append((new_chrom, fit))

    return  population

 # 적합도 계산
def eval_fit_distance(lst):
    global matrix
    chrm = copy.copy(lst)
    chrm.append(chrm[0])

    cost = 0
    for idx in range(len(chrm)-1):
        dist = matrix[chrm[idx]][chrm[idx+1]]
        cost += dist

    return cost

# 룰렛 휠 셀렉션
def roulette_wheel_selection(population):
    sol=[]
    max = sum(chrom[1] for chrom in population)
    pick = np.random.uniform(0,max,2)

    for i in range(2):
      current = 0
      for chrom in population:
        current += chrom[1]
        if (current > pick[i]):
          sol.append(chrom)
          break

    return sol[0], sol[1]

# cross_over
def cross_over(course1, course2):
    switch = random.randrange(0, len(course1))
    child = course1[:switch+1]
    child += [i for i in course2 if i not in child]
    return child

def order_cross_over(course1, course2):
    switch1, switch2 = random.sample(range(0, len(course1)), 2)
    if switch1 > switch2: switch1, switch2 = switch2, switch1
    child1 = course1[switch1:switch2+1]

    child2 = [i for i in course2[switch2+1:] if i not in child1]
    child2 += child1
    child2 += [i for i in course2[:switch2+1] if i not in child1]
    return child2

# mutation
def mutation(course):
    prob = 0.1
    if random.random() < prob:
        switch1, switch2 = random.sample(range(0, len(course)), 2)
        course[switch1], course[switch2] = course[switch2], course[switch1]
    return course

def inversion(course):
    prob = 0.1
    if random.random() < prob:
        switch1, switch2 = random.sample(range(0, len(course)), 2)
        inversion = course[switch1:switch2]
        inversion.reverse()
        course[switch1:switch2] = inversion[:]
    return course

# 대치
def replace_chrom(population, offspring, population_size):
    offspring_max = max(offspring, key=lambda chrom: chrom[1])
    population_min = min(population, key=lambda chrom: chrom[1])
    
    if offspring_max[1] > population_min[1]:
        population += offspring
        population.sort(key=lambda chrom: -chrom[1])
        del population[population_size:]

population_size = 10
offspring_size = 10
C = 1000000
offspring = []
total_dist=[]

# 1. 초기 모집단 생성
eval_matrix()
population = init_population_tree(population_size, C)

for idx in range(10001):
  offspring.clear()
  while len(offspring) != offspring_size:
    # 3. 룰렛 휠 선택
    chorm1, chorm2 = roulette_wheel_selection(population)

    # 4. cross over & mutation
    new_chrom = mutation(order_cross_over(chorm1[0], chorm2[0]))
    fit = pow(C/eval_fit_distance(new_chrom), 2)
    offspring.append((new_chrom, fit))

  # 5. replace
  replace_chrom(population, offspring, population_size)

  # 6. 각 세대 최적값 넣기
  total_dist.append(eval_fit_distance(population[0][0]))      

  # 출력
  if idx == 0:
    print(idx, " : 적합도", population[0][1], ",  cost ", total_dist[idx])

  if idx >= 1000 and idx % 1000 ==0:
    print(idx, " : 적합도", population[0][1], ",  cost ", total_dist[idx])

path = pd.DataFrame({"Path": population[0][0]})
path.to_csv("solution1.csv", index=None)
cost = pd.DataFrame({"cost": total_dist})
cost.to_csv("cost.csv", index=None)

0  : 적합도 129802.49203632864 ,  cost  2775.610263858476
1000  : 적합도 129802.49203632884 ,  cost  2775.6102638584744
2000  : 적합도 129802.49203632884 ,  cost  2775.6102638584744
3000  : 적합도 129802.49203632884 ,  cost  2775.6102638584744
4000  : 적합도 129802.49203632884 ,  cost  2775.6102638584744
5000  : 적합도 129802.49203632884 ,  cost  2775.6102638584744
6000  : 적합도 129802.49203632884 ,  cost  2775.6102638584744
7000  : 적합도 129802.49203632884 ,  cost  2775.6102638584744
8000  : 적합도 129802.49203632884 ,  cost  2775.6102638584744
9000  : 적합도 129802.49203632884 ,  cost  2775.6102638584744
10000  : 적합도 129802.49203632884 ,  cost  2775.6102638584744


In [6]:
print(path)

     Path
0     489
1     542
2     419
3      46
4     220
..    ...
995   746
996   189
997   221
998   154
999   621

[1000 rows x 1 columns]


In [9]:
# 트리구조 사용하지 않고 유전 알고리즘만 사용했을 때

population_size = 10
offspring_size = 10
C = 1000000
offspring = []
total_dist=[]

# 1. 초기 모집단 생성
eval_matrix()
population = init_population_random(population_size, C)

for idx in range(10001):
  offspring.clear()
  while len(offspring) != offspring_size:
    # 3. 룰렛 휠 선택
    chorm1, chorm2 = roulette_wheel_selection(population)

    # 4. cross over & mutation
    new_chrom = mutation(order_cross_over(chorm1[0], chorm2[0]))
    fit = pow(C/eval_fit_distance(new_chrom), 2)
    offspring.append((new_chrom, fit))

  # 5. replace
  replace_chrom(population, offspring, population_size)

  # 6. 각 세대 최적값 넣기
  total_dist.append(eval_fit_distance(population[0][0]))      

  # 출력
  if idx == 0:
    print(idx, " : 적합도", population[0][1], ",  cost ", total_dist[idx])

  if idx >= 1000 and idx % 1000 ==0:
    print(idx, " : 적합도", population[0][1], ",  cost ", total_dist[idx])

path = pd.DataFrame({"Path": population[0][0]})
path.to_csv("random_solution1.csv", index=None)
cost = pd.DataFrame({"cost": total_dist})
cost.to_csv("random_cost.csv", index=None)

0  : 적합도 387.8199586159662 ,  cost  50779.09098687922
1000  : 적합도 1204.0913037094151 ,  cost  28818.428202689996
2000  : 적합도 1875.304526485769 ,  cost  23092.13559921295
3000  : 적합도 2350.4251147671785 ,  cost  20626.559338272793
4000  : 적합도 2831.149864469588 ,  cost  18793.97178453641
5000  : 적합도 3287.159276433399 ,  cost  17441.73268063166
6000  : 적합도 3729.228882544627 ,  cost  16375.34582366128
7000  : 적합도 4179.308135592132 ,  cost  15468.485830099258
8000  : 적합도 4542.5983055865845 ,  cost  14837.059308762751
9000  : 적합도 4852.310233411884 ,  cost  14355.74449482043
10000  : 적합도 5165.049468156911 ,  cost  13914.344681056118
